<a href="https://colab.research.google.com/github/Izhaar-ahmed/MemoryPalAI/blob/main/MemoryPalAI.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install langchain langchain_community pypdf
!pip install langchain-openai

INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 51.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 323.6/323.6 kB 35.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.7/64.7 kB 7.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.32.4
    Uninstalling requests-2.32.4:
      Successfully uninstalled requests-2.32.4
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-colab 1.0.0 requires requests==2.32.4, but you have requests 2.32.5 which is incompatible.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.9/81.9 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━

In [2]:
import os
from getpass import getpass

# This sets your OpenRouter key as an environment variable
os.environ["OPENROUTER_API_KEY"] = getpass("Enter your OpenRouter API Key: ")

print("✅ OpenRouter API Key has been set.")

Enter your OpenRouter API Key: ··········
✅ OpenRouter API Key has been set.


In [4]:
!pip install -U openai-whisper
!sudo apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 803.2/803.2 kB 18.1 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for openai-whisper: filename=openai_whisper-20250625-py3-none-any.whl size=803979 sha256=064f9c9efc78f668f189929b5751e1c0445fc50f215f51518ff2ab93683ec4fa
  Stored in directory: /root/.cache/pip/wheels/61/d2/20/09ec9bef734d126cba375b15898010b6cc28578d8afdde5869
Successfully built openai-whisper
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 38 not upgraded.


In [5]:
import os
import whisper
from langchain_community.document_loaders import TextLoader, PyPDFLoader
from langchain_core.documents import Document # We need this to wrap our transcribed text

class IngestionAgent:
    """
    An agent responsible for ingesting data from various sources.
    It detects the file type and uses the appropriate loader or model.
    """
    def __init__(self):
        # Load the whisper model once when the agent is created for efficiency.
        # 'base' is a good balance of speed and accuracy for testing.
        self.whisper_model = whisper.load_model("base")
        print("✅ Whisper model loaded into IngestionAgent.")

    def ingest(self, file_path: str):
        """
        Loads a document from a file path and extracts its content.
        For audio files, it transcribes them to text.
        """
        _, file_extension = os.path.splitext(file_path)
        print(f"\n🚀 Ingestion Agent: Processing file '{file_path}'...")

        # Define supported audio extensions
        audio_extensions = ['.mp3', '.m4a', '.wav', '.mpeg']

        if file_extension.lower() == '.txt':
            loader = TextLoader(file_path)
            documents = loader.load()
        elif file_extension.lower() == '.pdf':
            loader = PyPDFLoader(file_path)
            documents = loader.load()
        elif file_extension.lower() in audio_extensions:
            # --- This is the new logic for audio files ---
            print("🔊 Audio file detected. Transcribing with Whisper...")
            result = self.whisper_model.transcribe(file_path)
            transcribed_text = result['text']

            # We must wrap the transcribed text in a LangChain 'Document' object
            # to keep our output format consistent with the other loaders.
            metadata = {"source": file_path}
            documents = [Document(page_content=transcribed_text, metadata=metadata)]
        else:
            print(f"⚠️ Ingestion Agent: File type '{file_extension}' not supported.")
            return None

        print(f"✅ Ingestion Agent: Successfully processed content.")
        return documents

print("✅ IngestionAgent class has been upgraded to handle audio.")

✅ IngestionAgent class has been upgraded to handle audio.


In [10]:
import shutil

# --- 1. DEFINE THE PATHS ---
# This is the source file with spaces in its path
source_path = "/content/drive/My Drive/MemoryPalAI_Project/harvard.wav"

# This is a new, simple path in the local Colab runtime
local_path = "/content/temp_harvard.wav"

# --- 2. COPY THE FILE ---
# Copy the file from Google Drive to the simple local path
try:
    shutil.copy(source_path, local_path)
    print(f"✅ File successfully copied to {local_path}")

    # --- 3. TEST THE AGENT WITH THE NEW, SIMPLE PATH ---
    # Create an instance of our upgraded agent
    upgraded_agent = IngestionAgent()

    # --- Test Case: Ingesting the 'harvard.wav' audio file ---
    audio_documents = upgraded_agent.ingest(local_path)

    if audio_documents:
        print("\n--- Transcribed Content from 'harvard.wav' ---")
        print(audio_documents[0].page_content)
        print("------------------------------------------")

except FileNotFoundError:
    print(f"❌ ERROR: Could not find the source file. Make sure your Google Drive is mounted.")
except Exception as e:
    print(f"❌ ERROR: An error occurred. {e}")

✅ File successfully copied to /content/temp_harvard.wav
✅ Whisper model loaded into IngestionAgent.

🚀 Ingestion Agent: Processing file '/content/temp_harvard.wav'...
🔊 Audio file detected. Transcribing with Whisper...
✅ Ingestion Agent: Successfully processed content.

--- Transcribed Content from 'harvard.wav' ---
 The stale smell of old beer lingers. It takes heat to bring out the odor. A cold dip restores health and zest. A salt pickle tastes fine with ham. Tacos al pastor are my favorite. A zestful food is the hot cross bun.
------------------------------------------


In [12]:
# Define the permanent path to your file in Google Drive
pdf_path = "/content/drive/My Drive/MemoryPalAI_Project/01_Agentic AI_Overview of AI and GenAI.pdf"

# --- Create an instance of the agent ---
# This is the line that was missing
ingestion_agent = IngestionAgent()

# --- Test Case: Ingesting your uploaded PDF file ---
pdf_documents = ingestion_agent.ingest(pdf_path)

if pdf_documents:
    # PDFs can have multiple pages. Let's print the content of the first page.
    print("\n--- Content from the first page of the PDF ---")
    print(pdf_documents[0].page_content)
    print("------------------------------------------")

✅ Whisper model loaded into IngestionAgent.

🚀 Ingestion Agent: Processing file '/content/drive/My Drive/MemoryPalAI_Project/01_Agentic AI_Overview of AI and GenAI.pdf'...
✅ Ingestion Agent: Successfully processed content.

--- Content from the first page of the PDF ---
Overview of AI and Generative AI
Dr. Sunil Saumya
Dept of Data Science & Artificial Intelligence
Indian Institute of Information Technology Dharwad
sunilsaumya @iiitdwd.ac.in
August 6, 2025
Dr. Sunil Saumya (IIIT Dharwad) Agentic AI August 6, 2025 1 / 32
------------------------------------------


In [13]:
import os
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
import json

class OrganizerAgent:
    """
    An agent that analyzes text to extract entities and relationships
    to build a knowledge graph.
    """
    def __init__(self):
        # We will use OpenRouter to access a free model.
        self.llm = ChatOpenAI(
            # This is the FIX with the correct model ID from the website.
            model="tngtech/deepseek-r1t-chimera:free",
            base_url="https://openrouter.ai/api/v1",
            api_key=os.environ.get("OPENROUTER_API_KEY")
        )
        print("✅ OrganizerAgent initialized with LLM.")

    def extract_graph_data(self, text: str):
        """
        Uses an LLM to extract nodes (entities) and edges (relationships) from text.
        """
        print("\n🚀 Organizer Agent: Analyzing text to extract graph data...")

        parser = JsonOutputParser()

        prompt = ChatPromptTemplate.from_template(
            """
            You are an expert at creating knowledge graphs from text.
            Analyze the following text and extract the key entities as nodes and the relationships between them as edges.

            RULES:
            - Nodes must have an 'id' (a unique name) and a 'type' (e.g., 'Person', 'Concept', 'Organization').
            - Edges must have a 'source' id, a 'target' id, and a 'label' describing their relationship.
            - Only extract relationships explicitly mentioned in the text.

            TEXT:
            {text_input}

            {format_instructions}
            """
        )

        chain = prompt | self.llm | parser

        try:
            graph_data = chain.invoke({
                "text_input": text,
                "format_instructions": parser.get_format_instructions(),
            })
            print("✅ Organizer Agent: Successfully extracted graph data.")
            return graph_data
        except Exception as e:
            print(f"❌ Organizer Agent: Failed to extract data. Error: {e}")
            return None

# --- Let's test the new agent with the text from your PDF ---

organizer = OrganizerAgent()

pdf_text = """
Overview of AI
Dr. Sunil Saumya
Dept of Data Science & Artificial Intelligence
Indian Institute of Information Technology Dharwad
sunilsaumya @iiitdwd.ac.in
August 6, 2025
Dr. Sunil Saumya (IIIT Dharwad) Agentic AI August 6, 2025 1 / 32
"""

knowledge_graph_data = organizer.extract_graph_data(pdf_text)

if knowledge_graph_data:
    print("\n--- Extracted Knowledge Graph ---")
    print(json.dumps(knowledge_graph_data, indent=2))
    print("---------------------------------")

✅ OrganizerAgent initialized with LLM.

🚀 Organizer Agent: Analyzing text to extract graph data...
✅ Organizer Agent: Successfully extracted graph data.

--- Extracted Knowledge Graph ---
{
  "nodes": [
    {
      "id": "Dr. Sunil Saumya",
      "type": "Person"
    },
    {
      "id": "Dept of Data Science & Artificial Intelligence",
      "type": "Organization"
    },
    {
      "id": "IIIT Dharwad",
      "type": "Organization"
    },
    {
      "id": "AI",
      "type": "Concept"
    },
    {
      "id": "Generative AI",
      "type": "Concept"
    },
    {
      "id": "Agentic AI",
      "type": "Concept"
    }
  ],
  "edges": [
    {
      "source": "Dr. Sunil Saumya",
      "target": "Dept of Data Science & Artificial Intelligence",
      "label": "affiliated with"
    },
    {
      "source": "Dept of Data Science & Artificial Intelligence",
      "target": "IIIT Dharwad",
      "label": "part of"
    },
    {
      "source": "Dr. Sunil Saumya",
      "target": "IIIT Dharwa